### 모듈 불러오기

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
import gc
from six.moves import urllib
warnings.filterwarnings('ignore')

In [22]:
# Scalers
from sklearn.preprocessing import StandardScaler
    # 입력된 1개 이상의 배열을 동일한 순서로 섞어준다.
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
    # 여러 모델 변환 개체의 결과를 연결한다.
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from scipy.stats import reciprocal, uniform

from sklearn.ensemble import AdaBoostClassifier

# Cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Common data processors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from scipy import sparse

# Accuracy Score
from sklearn.metrics import accuracy_score

In [23]:
np.random.seed(123)

%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
train = pd.read_csv('./data/train_users_2.csv')
test = pd.read_csv('./data/test_users.csv')

In [25]:
id_test = test['id']
labels = train['country_destination'].values
df_train = train.drop(['country_destination'], axis=1)
#변수명 맘에 듬...train_flag
train_flag = df_train.shape[0]

In [3]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [15]:
print(f'train columns is {len(train.columns)}', f'test columns is {len(test.columns)}')

train columns is 16 test columns is 15


In [19]:
print(f'train shape is {train.shape}',
      f'test shape is {test.shape}')


train shape is (213451, 16) test shape is (62096, 15)


In [8]:
set(train.columns) - set(test.columns)

{'country_destination'}

In [16]:
train['country_destination'].value_counts()

NDF      124543
US        62376
other     10094
FR         5023
IT         2835
GB         2324
ES         2249
CA         1428
DE         1061
NL          762
AU          539
PT          217
Name: country_destination, dtype: int64

In [18]:
train['country_destination'].value_counts(normalize=True)

NDF      0.583473
US       0.292226
other    0.047290
FR       0.023532
IT       0.013282
GB       0.010888
ES       0.010536
CA       0.006690
DE       0.004971
NL       0.003570
AU       0.002525
PT       0.001017
Name: country_destination, dtype: float64

In [17]:
train['country_destination'].nunique()

12

In [26]:
df_total = pd.concat([df_train, test], axis=0, ignore_index=True)

In [27]:
df_total.shape

(275547, 15)

In [31]:
# na값이 너무 많아서 빼는듯?!
df_total['date_first_booking'].isna().sum()

186639

In [32]:
df_total = df_total.drop(['id','date_first_booking'], axis=1)

In [65]:
# vstack <- 열의 개수가 같은 배열들을 세로로 합쳐준다.
date_ac = np.vstack(df_total['date_account_created'].astype(str).apply(lambda x:list(map(int, x.split('-')))).values)

In [63]:
#일자가 str형태라 split후 map을 사용하여 전부 int화
df_total['date_account_created'].astype(str).apply(
    lambda x: list(map(int, x.split('-'))))

0         [2010, 6, 28]
1         [2011, 5, 25]
2         [2010, 9, 28]
3         [2011, 12, 5]
4         [2010, 9, 14]
              ...      
275542    [2014, 9, 30]
275543    [2014, 9, 30]
275544    [2014, 9, 30]
275545    [2014, 9, 30]
275546    [2014, 9, 30]
Name: date_account_created, Length: 275547, dtype: object

In [69]:
df_total['Day'] = date_ac[:,0]
df_total['Month'] = date_ac[:,1]
df_total['year'] = date_ac[:,2]

df_total = df_total.drop(['date_account_created'], axis=1)

In [76]:
time_stp = np.vstack(df_total['timestamp_first_active'].astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:]]))).values)
time_stp

array([[  2009,      3,     19,  43255],
       [  2009,      5,     23, 174809],
       [  2009,      6,      9, 231247],
       ...,
       [  2014,      9,     30, 235408],
       [  2014,      9,     30, 235430],
       [  2014,      9,     30, 235901]])

In [78]:
df_total['tfa_day'] = time_stp[:,0]
df_total['tfa_Month'] = time_stp[:,1]
df_total['tfa_year'] = time_stp[:, 2]

df_total = df_total.drop(['timestamp_first_active'], axis=1)

In [87]:
df_total.loc[((df_total['age'] < 14) | (df_total['age'] > 100)),'age'] = -1

In [93]:
df_total['age'].max()

100.0

### OHE